In [1]:
from sklearn.datasets import fetch_california_housing

In [2]:
california = fetch_california_housing()

In [ ]:
print(california.DESCR)

In [4]:
x = california.data
y = california.target

In [5]:
x.shape, y.shape

((20640, 8), (20640,))

* 조건 : **Functional API**
    1. training set / validation set / test set 분리! (자율)
    2. hidden layer 최소 3개 사용! (노드 수 자율)
    3. early stopping 적용! (patience 7)
    4. validation_data 옵션에 validation set 적용! 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_x, test_x, train_y, test_y =\
    train_test_split(x, y, test_size=0.2, random_state=2023)

train_x, val_x, train_y, val_y =\
    train_test_split(train_x, train_y, test_size=0.2, random_state=2023)

In [8]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((13209, 8), (13209,), (3303, 8), (3303,), (4128, 8), (4128,))

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
## 모델링 : Functional API
# 1번 세션 클리어
keras.backend.clear_session()

# 2번 레이어 사슬처럼 엮기
il = keras.layers.Input(shape=(8,))
hl = keras.layers.Dense(512, activation='relu')(il)
hl = keras.layers.Dense(256, activation='relu')(hl)
hl = keras.layers.Dense(128, activation='relu')(hl)
ol = keras.layers.Dense(1)(hl)

# 3번 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

# 4번 컴파일
model.compile(loss='mse', optimizer='adam')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 512)               4608      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 168,961
Trainable params: 168,961
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
es = EarlyStopping(monitor='val_loss',         # 관측 대상
                   min_delta=0,                # 관측 대상의 임계값 설정
                   patience=5,                 # 관측 대상의 성능이 증가하지 않았을 때, 얼마나 더 지켜볼래
                   verbose=1,
                   restore_best_weights=True)  # 최적 epoch 시점의 가중치로 설정할래!

In [13]:
# 학습
model.fit(train_x, train_y, epochs=1000, verbose=1,
          callbacks=[es], validation_data=(val_x, val_y) )

Epoch 1/1000
413/413 [==============================] - 4s 7ms/step - loss: 242.1615 - val_loss: 2.7596
Epoch 2/1000
413/413 [==============================] - 3s 6ms/step - loss: 3.4422 - val_loss: 1.0598
Epoch 3/1000
413/413 [==============================] - 3s 8ms/step - loss: 29.4680 - val_loss: 1.2627
Epoch 4/1000
413/413 [==============================] - 3s 6ms/step - loss: 3.0992 - val_loss: 1.0747
Epoch 5/1000
413/413 [==============================] - 3s 6ms/step - loss: 1.5030 - val_loss: 0.8658
Epoch 6/1000
413/413 [==============================] - 3s 6ms/step - loss: 15.4420 - val_loss: 1.2999
Epoch 7/1000
413/413 [==============================] - 3s 8ms/step - loss: 1.7729 - val_loss: 1.1686
Epoch 8/1000
413/413 [==============================] - 3s 6ms/step - loss: 4.7023 - val_loss: 0.8420
Epoch 9/1000
413/413 [==============================] - 3s 6ms/step - loss: 0.8672 - val_loss: 1.2218
Epoch 10/1000
413/413 [==============================] - 3s 6ms/step - loss: 0

In [15]:
# 예측값 생성
y_pred = model.predict(test_x)

129/129 [==============================] - 0s 2ms/step


In [16]:
y_pred[:10].reshape(-1)

array([1.4880791, 2.212721 , 1.4423921, 1.173657 , 1.7522199, 1.9928014,
       1.8559041, 1.8538053, 1.4676633, 2.1741292], dtype=float32)

In [18]:
test_y[:10]

array([3.656, 2.01 , 2.132, 0.986, 1.53 , 1.326, 2.125, 3.068, 1.573,
       1.766])